In [9]:
import os
import sys
sys.path.append('../')  # subir 1 nivel desde notebooks a src
from src.config.settings import COUNTRY, SRC_DIR

In [10]:
from src.sources.inventory.app import Inventory
db_loader = Inventory()
df_data_base = db_loader.load_db_from_country_selected()

CO


c:\Users\JTRUJILLO\Documents\Galgo\Scripts\Otros\deep_research_models\notebooks\..\src\sources\inventory\app.py:23: DtypeWarning: Columns (8,23,30) have mixed types. Specify dtype option on import or set low_memory=False.
  data_base = pd.read_csv(CO_INVENTORY_PATH)


In [11]:
models_list = [
    "CO731-tvs-raider-125",
    "CO928-akt-motos-nkd-125",
    # "CO2961-hero-hunk-125-r",
    # "CO2861-tvs-raider-fi-125",
    # "CO1033-bajaj-boxer-ct-100-es",
    # "CO1996-hero-hunk-160r-4v",
    # "CO736-tvs-apache-rtr-160-4v-xc-fi",
    # "CO734-tvs-apache-rtr-160-4v-xconnect",
    # "CO1480-hero-eco-deluxe-cw-c-2024",
    # "CO569-tvs-neo-nx",
    # "CO570-tvs-raider-125-racing" # Problemática
]

In [12]:
from src.utils.datetime_utils import get_current_date_format_yyyy_mm_dd
from src.core.base_knowledge import get_basic_data_model
from src.core.prompt_utils import *
from src.core.gemini_processor import GeminiProcessor
gemini_processor = GeminiProcessor()

In [13]:
knowledge_base_final = ""

for idx, model in enumerate(models_list):
    df_modelo = df_data_base[df_data_base["code"] == model]
    data_model = get_basic_data_model(df_modelo, COUNTRY)

    header = f"""=================================================
CODE = {data_model["code"]} | {data_model["marca"]} {data_model["modelo"]}
=================================================
"""

    basic_data_model_info = f""" [DATOS BASICOS]
Marca: {data_model["marca"]}
Modelo: {data_model["modelo"]}
Año: {data_model["año"]}
Pais: {data_model["pais"]}
Tipo: {data_model["tipo"]}
URL: {data_model["url"]}
    """

    footer = f"""================================================="""

    # Leer plantilla del prompt
    prompt_from_file = read_prompt_from_file(f"../src/data/input/prompts/knowledge_base_template.md")


    # Reemplazar variables
    first_research_prompt = replace_variables(prompt_from_file, {
        "{marca}": data_model["marca"],
        "{modelo}": data_model["modelo"],
        "{año}": str(data_model["año"]),
        "{pais}": COUNTRY,
        "{tipo}": data_model["tipo"]
        })

    deep_research = gemini_processor.send_prompt(first_research_prompt)

    # Footer: doble separador entre modelos
    if idx < len(models_list) - 1:  # No es el último modelo
        footer = f"""=================================================
=================================================
"""
    else:  # Es el último modelo
        footer = f"""=================================================
================================================="""

    # Construir knowledge base de este modelo
    knowledge_base = header + "\n" + basic_data_model_info + "\n" + deep_research + "\n" + footer + "\n"
    knowledge_base_final += knowledge_base

In [14]:
print(knowledge_base_final)

CODE = CO731-tvs-raider-125 | TVS Raider 125

 [DATOS BASICOS]
Marca: TVS
Modelo: Raider 125
Año: 2026
Pais: CO
Tipo: ['Urbana']
URL: https://www.galgo.com/co/motos/CO731-tvs-raider-125
    
Esta investigación ha sido realizada asumiendo que el **{país} es Colombia**, dado que es el mercado donde este modelo tiene mayor tracción bajo la configuración de año "2026" (modelos adelantados) y donde el contexto de "recordar ubicación" es relevante.

**Deep Sentiment Research: TVS Raider 125 (Modelo 2026 - Mercado Colombia)**

[SEGMENTO]
**Tipo según BD:** ['Urbana']
**Segmento identificado por usuarios:** Los usuarios colombianos la elevan a una categoría de **"Premium Entry-Level"** o "Sport Commuter". No se percibe como una moto de trabajo (tipo Boxer o NKD), sino como una moto de estilo de vida y transporte personal universitario/oficina.

[SENTIMIENTO]
En Colombia, la TVS Raider 125 es percibida como la **reina tecnológica** de su categoría. El sentimiento general es de orgullo: el propi

In [15]:
with open(f"../src/data/output/gemini/{COUNTRY}-knowledge_base_final.md", "w", encoding="utf-8") as f:
    f.write(knowledge_base_final)